# Prétraitement des données scrapées

In [1]:
# Importer les bibliothèques nécessaires
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer

## 1. Importer les données scrapées

In [2]:
data_path = "../data/raw_data/autohero.csv  "
df = pd.read_csv(data_path, sep=",", encoding="utf-8")

In [3]:
df.head()

,scraped_at,modele,finition,prix,annee_mise_en_circulation,kilometrage,carburant,transmission,puissance,nb_ancien_proprietaire,classe_vehicule,nb_porte,nb_place,couleur,sellerie,classe_emission,emission_CO2,crit_air,usage_commerciale_anterieure,url_annonce
0,2025-04-09,Ford Fiesta,1.0 EcoBoost ST-Line X,13 190 €,15.05.2020,69 301 km,Essence,Boite de vitesse manuelle,95 CV / 70 kW,3,Citadine,5.0,5.0,Gris,Tissu (Sellerie d'origine),EURO 6,NaN,Crit'Air 1,NaN,https://www.autohero.com/fr/ford-fiesta/id/516...
1,2025-04-09,Toyota ProAce,Combi Long 1.5 D-4D Dynamic,23 990 €,29.04.2021,71 887 km,Diesel,Boite de vitesse manuelle,120 CV / 88 kW,2,Monospace,4.0,9.0,Gris,Tissu (Sellerie d'origine),EURO 6,170 g/km,Crit'Air 2,Oui,https://www.autohero.com/fr/toyota-pro-ace/id/...
2,2025-04-09,Mercedes-Benz GLA,250 e AMG Line 8G-DCT,32 490 €,23.10.2020,59 649 km,Hybride,Double embrayage / DCT,218 CV / 160 kW,2,SUV,5.0,5.0,Gris,Mi-cuir (Sellerie d'origine),EURO 6,32 g/km,Crit'Air 1,Oui,https://www.autohero.com/fr/mercedes-benz-gla/...
3,2025-04-09,BMW X1,sDrive18i xLine DKG7,27 890 €,05.05.2021,37 869 km,Essence,Double embrayage / DCT,136 CV / 100 kW,2,SUV,5.0,5.0,Noir,Mi-cuir (Sellerie d'origine),EURO 6,148 g/km,Crit'Air 1,Non,https://www.autohero.com/fr/bmw-x-1/id/490f203...
4,2025-04-09,Peugeot 3008,1.5 Blue-HDi Crossway EAT8,19 090 €,31.12.2019,58 958 km,Diesel,Boite de vitesse automatique,130 CV / 96 kW,3,SUV,5.0,5.0,Blanc,Mi-cuir (Sellerie d'origine),EURO 6,98 g/km,Crit'Air 2,Oui,https://www.autohero.com/fr/peugeot-3008/id/df...


In [4]:
df.shape

(2352, 20)

Il y a 2352 lignes (annonces) et 22 colonnes

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2352 entries, 0 to 2351
Data columns (total 20 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   scraped_at                    2352 non-null   object 
 1   modele                        2342 non-null   object 
 2   finition                      2342 non-null   object 
 3   prix                          2342 non-null   object 
 4   annee_mise_en_circulation     2342 non-null   object 
 5   kilometrage                   2342 non-null   object 
 6   carburant                     2342 non-null   object 
 7   transmission                  2342 non-null   object 
 8   puissance                     2342 non-null   object 
 9   nb_ancien_proprietaire        2342 non-null   object 
 10  classe_vehicule               2342 non-null   object 
 11  nb_porte                      2342 non-null   float64
 12  nb_place                      2342 non-null   float64
 13  cou

## 2. Pré transformation des données

In [6]:
# Convertir la colonne "prix" en float
# Supprimer les caractères indésirables et convertir en float
df["prix"] = (df["prix"]
              .str.replace("€", "")
              .str.replace("\u202f", "")
              .str.replace(" ", "")
              .str.replace(",", ".")
              .astype(float))

# Convertir la colonne "kilométrage" en float
# Supprimer les caractères indésirables et convertir en float
df["kilometrage"] = (df["kilometrage"]
                     .str.replace("km", "")
                     .str.replace("\u202f", "")
                     .str.replace(" ", "")
                     .astype(float))

# Extraire l'année de mise en circulation et la convertir en int
df["annee"] = (df["annee_mise_en_circulation"]
               .str.extract(r'(\d{4})')[0]
               .astype('Int64'))
                                   
# Convertir la colonne "puissance" en puissance_cv
# Extraire uniquement les chiffres avant "CV"
df["puissance"] = (df["puissance"]
                      .str.extract(r'(\d+) CV')[0]
                      .astype(float))

# Convertir la colonne "nb_porte" en int
df["nb_porte"] = df["nb_porte"].astype('Int64')

# Convertir la colonne "nb_place" en int
df["nb_place"] = df["nb_place"].astype('Int64')

# Extraire l'émission CO2 et la convertir en float
df["emission_CO2"] = (df["emission_CO2"]
                      .str.extract(r'(\d+) g/km')[0]
                      .astype(float))


In [7]:
df[df['prix'].isnull()]

,scraped_at,modele,finition,prix,annee_mise_en_circulation,kilometrage,carburant,transmission,puissance,nb_ancien_proprietaire,...,nb_porte,nb_place,couleur,sellerie,classe_emission,emission_CO2,crit_air,usage_commerciale_anterieure,url_annonce,annee
871,2025-04-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,https://www.autohero.com/fr/bmw-serie-1/id/599...,<NA>
1007,2025-04-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,https://www.autohero.com/fr/toyota-yaris/id/26...,<NA>
1082,2025-04-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,https://www.autohero.com/fr/citroen-c-3/id/988...,<NA>
1561,2025-04-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,https://www.autohero.com/fr/hyundai-tucson/id/...,<NA>
1752,2025-04-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,https://www.autohero.com/fr/kia-x-ceed/id/0647...,<NA>
1874,2025-04-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,https://www.autohero.com/fr/renault-captur/id/...,<NA>
1942,2025-04-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,https://www.autohero.com/fr/hyundai-i-10/id/4d...,<NA>
2047,2025-04-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,https://www.autohero.com/fr/renault-clio/id/e3...,<NA>
2158,2025-04-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,https://www.autohero.com/fr/audi-q-2/id/a9a82f...,<NA>
2327,2025-04-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,https://www.autohero.com/fr/volkswagen-golf-vi...,<NA>


In [8]:
# Calculer l'âge de la voiture:
df["age_days"] = (pd.to_datetime(df["scraped_at"]) -
                  pd.to_datetime(df["annee_mise_en_circulation"], dayfirst=True))
df["age_days"] = df["age_days"].apply(lambda x: x.days if pd.notnull(x) else None)

df["age_years"] = df["age_days"] / 365  # en années
df["age_years"] = df["age_years"].apply(lambda x: round(x, 1) if pd.notnull(x) else None)

In [9]:
# Extraire la marque de la voiture et l'insérer à côté de la colonne "modele"
# en utilisant la première partie de la chaîne avant l'espace
df.insert(df.columns.get_loc("modele"), "marque", df["modele"].str.split(" ").str[0])

In [10]:
observations_by_modele_annee = df.groupby(['modele', 'annee']).size().reset_index(name='count')
observations_by_modele_annee.sort_values(by= 'count', ascending = False)

,modele,annee,count
557,Renault Clio,2019,34
498,Peugeot 2008,2020,33
497,Peugeot 2008,2019,28
508,Peugeot 208,2022,21
513,Peugeot 3008,2019,21
...,...,...,...
9,Alfa Romeo Giulietta,2017,1
8,Alfa Romeo Giulia,2021,1
7,Alfa Romeo Giulia,2020,1
6,Abarth 595C,2020,1


In [11]:
observations_by_modele = df.groupby(['modele']).size().reset_index(name='count')
observations_by_modele.sort_values(by= 'count', ascending = False)

,modele,count
151,Peugeot 2008,115
163,Renault Clio,100
152,Peugeot 208,88
153,Peugeot 3008,82
32,Citroen C3,63
...,...,...
197,Toyota ProAce,1
208,Volkswagen Passat,1
214,Volkswagen Tiguan Allspace,1
218,Volvo V40 Cross Country,1


## 3. Récupérer le prix neuf

In [12]:
# Importer la data des prix neufs
df_prix_neuf = pd.read_csv("../data/processed_data/prix_neuf_voitures_vf.csv", sep=",", encoding="utf-8")

In [13]:
df_prix_neuf

,source_model,Versions,Portes,Energie,Boite,CO2\n(g/km),Prix,url,option_marque_select,option_year_select,Version_selected
0,Abarth 124 Spider,II 1.4 TURBO 170,2.0,Ess.,Mécanique,146 (nedc),34 500 €,https://www.caradisiac.com/fiches-techniques/m...,ABARTH,2019,abarth 124 2e generation spider
1,Abarth 124 Spider,II 1.4 TURBO 170 GT,2.0,Ess.,Mécanique,146 (nedc),40 900 €,https://www.caradisiac.com/fiches-techniques/m...,ABARTH,2019,abarth 124 2e generation spider
2,Abarth 124 Spider,II 1.4 TURBO 170 GT BVA,2.0,Ess.,Automatique,161 (nedc),42 900 €,https://www.caradisiac.com/fiches-techniques/m...,ABARTH,2019,abarth 124 2e generation spider
3,Abarth 124 Spider,II 1.4 TURBO 170 TURISMO,2.0,Ess.,Mécanique,146 (nedc),37 500 €,https://www.caradisiac.com/fiches-techniques/m...,ABARTH,2019,abarth 124 2e generation spider
4,Abarth 124 Spider,II 1.4 TURBO 170 TURISMO BVA,2.0,Ess.,Automatique,161 (nedc),39 500 €,https://www.caradisiac.com/fiches-techniques/m...,ABARTH,2019,abarth 124 2e generation spider
...,...,...,...,...,...,...,...,...,...,...,...
56684,S-CROSS,1.4 BOOSTERJET HYBRID PRIVILEGE AUTO,5.0,Ess.,Automatique,129 (wltp),27 990 €,https://www.caradisiac.com/fiches-techniques/m...,SUZUKI,2021,suzuki s cross
56685,S-CROSS,1.4 BOOSTERJET HYBRID STYLE,5.0,Ess.,Mécanique,122 (wltp),32 240 €,https://www.caradisiac.com/fiches-techniques/m...,SUZUKI,2021,suzuki s cross
56686,S-CROSS,1.4 BOOSTERJET HYBRID STYLE 2020,5.0,Ess.,Mécanique,123 (wltp),27 640 €,https://www.caradisiac.com/fiches-techniques/m...,SUZUKI,2021,suzuki s cross
56687,S-CROSS,1.4 BOOSTERJET HYBRID STYLE ALLGRIP,5.0,Ess.,Mécanique,133 (wltp),29 640 €,https://www.caradisiac.com/fiches-techniques/m...,SUZUKI,2021,suzuki s cross


In [15]:
df_prix_neuf[(df_prix_neuf['source_model'].str.contains("Fiat 500X")) & (df_prix_neuf["option_year_select"]==2017)]

,source_model,Versions,Portes,Energie,Boite,CO2\n(g/km),Prix,url,option_marque_select,option_year_select,Version_selected
9094,Fiat 500X,II (2) 0.9 8V 105 TWINAIR S/S ANNIVERSARIO,3.0,Ess.,Mécanique,99 (nedc),18 190 €,https://www.caradisiac.com/fiches-techniques/m...,FIAT,2017,fiat 500 2e generation
9095,Fiat 500X,II (2) 0.9 8V 105 TWINAIR S/S CLUB,3.0,Ess.,Mécanique,99 (nedc),19 390 €,https://www.caradisiac.com/fiches-techniques/m...,FIAT,2017,fiat 500 2e generation
9096,Fiat 500X,II (2) 0.9 8V 105 TWINAIR S/S LOUNGE,3.0,Ess.,Mécanique,99 (nedc),17 890 €,https://www.caradisiac.com/fiches-techniques/m...,FIAT,2017,fiat 500 2e generation
9097,Fiat 500X,II (2) 0.9 8V 105 TWINAIR S/S RIVA,3.0,Ess.,Mécanique,99 (nedc),20 890 €,https://www.caradisiac.com/fiches-techniques/m...,FIAT,2017,fiat 500 2e generation
9098,Fiat 500X,II (2) 0.9 8V 105 TWINAIR S/S RIVA AVEC VOLANT...,3.0,Ess.,Mécanique,99 (nedc),21 740 €,https://www.caradisiac.com/fiches-techniques/m...,FIAT,2017,fiat 500 2e generation
...,...,...,...,...,...,...,...,...,...,...,...
9269,Fiat 500X,II (2) C 1.3 16V 95 MULTIJET S/S POPSTAR,2.0,Dies.,Mécanique,89 (nedc),20 140 €,https://www.caradisiac.com/fiches-techniques/m...,FIAT,2017,fiat 500 c
9270,Fiat 500X,II (2) C 1.3 16V 95 MULTIJET S/S RIVA,2.0,Dies.,Mécanique,89 (nedc),24 040 €,https://www.caradisiac.com/fiches-techniques/m...,FIAT,2017,fiat 500 c
9271,Fiat 500X,II (2) C 1.3 16V 95 MULTIJET S/S RIVA AVEC VOL...,2.0,Dies.,Mécanique,89 (nedc),24 890 €,https://www.caradisiac.com/fiches-techniques/m...,FIAT,2017,fiat 500 c
9272,Fiat 500X,II (2) C 1.3 16V 95 MULTIJET S/S S,2.0,Dies.,Mécanique,89 (nedc),21 840 €,https://www.caradisiac.com/fiches-techniques/m...,FIAT,2017,fiat 500 c


In [14]:
# Rename "marque" to match with the right name used in the caradisiac website (df_prix_neuf)
# Create a dictionary to map the names of the brands in df to the names used in df_prix_neuf
dict_name_marque = {'Abarth' : 'Abarth', 'Alfa' : 'Alfa Romeo', 'Audi' : 'Audi', 
                    'BMW' : 'BMW', 
                    'Citroen' : 'Citroen', 'Cupra' : 'Cupra', 
                    'DS' : 'DS', 'Dacia' : 'Dacia',
                    'Fiat' : 'Fiat', 'Ford' : 'Ford', 
                    'Honda' : 'Honda', 'Hyundai' : 'Hyundai', 
                    'Infiniti' : 'Infiniti', 
                    'Jaguar' : 'Jaguar', 'Jeep' : 'Jeep',
                    'Kia' : 'Kia', 
                    'Land' : 'Land Rover', 'Lexus' : 'Lexus', 
                    'MG' : "MG", 'MINI' : 'MINI', 'Mazda' : 'Mazda', 'Mercedes-Benz' : 'Mercedes', 'Mitsubishi' : 'Mitsubishi', 
                    'Nissan' : 'Nissan', 
                    'Opel' : 'Opel', 
                    'Peugeot' : 'Peugeot', 
                    'Renault' : 'Renault', 
                    'Seat' : 'Seat', 'Skoda' : 'Skoda', 'Smart' : 'Smart', 'Suzuki' : 'Suzuki', 
                    'Toyota' : 'Toyota', 
                    'Volkswagen' : 'Volkswagen', 'Volvo' : 'Volvo'
                    }

# Replace the names in the "marque" column of df with the corresponding names in df_prix_neuf
# using the dictionary created above
df['marque'] = df['marque'].replace(dict_name_marque)

In [ ]:
# Rename "marque" Citroen = DS if "modele" contains DS and "marque" contains Citroen
df.loc[(df['marque'].str.lower().str.contains('citroen', na=False)) & (df['modele'].str.lower().str.contains('ds', na=False)), "marque"] = "DS"

In [16]:
# Checking again, should be empty
df[(df['modele'].str.lower().str.contains("ds", na=False)) & (df['marque'].str.lower().str.contains('citroen', na=False))]

,scraped_at,marque,modele,finition,prix,annee_mise_en_circulation,kilometrage,carburant,transmission,puissance,...,couleur,sellerie,classe_emission,emission_CO2,crit_air,usage_commerciale_anterieure,url_annonce,annee,age_days,age_years


In [17]:
# Corriger manuellement le nom du modèle/marque pour qu'il soit aligné avec celui utilisé par le site caradisiac
# Créer un dictionnaire pour mapper les noms de modèles dans df à ceux utilisés dans df_prix_neuf
dict_model_corr = {
 'Abarth 595' : '500',
 'Abarth 595C' : '500',
 'Citroen C4 Grand Picasso' : 'C4 PICASSO',
 'Citroen C4 Picasso' : 'C4 PICASSO', 
 'Citroen DS3' : 'DS3',
 'Citroen DS3 Cabrio' : 'DS3',
 'Citroen DS4' : 'DS4',
 'Citroen DS4 Crossback' : 'DS4 CROSSBACK',
 'Citroen DS5' : 'DS5',
 'Ford Ka+' : 'KA+',
 'Ford Tourneo' : 'TOURNEO COURIER',
 "Kia cee'd" : 'CEED',
 "Kia pro_cee'd": 'PROCEED',
 'Land Rover Evoque' : 'RANGE ROVER EVOQUE',
 'Mercedes-Benz Classe GLB' : 'GLB',
 'Mercedes-Benz Classe GLC' : 'GLC',
 'Mercedes-Benz Classe GLE' : 'GLE',
 'Opel Crossland X' : 'CROSSLAND' ,
 'Suzuki SX4 S-Cross' : 'S-CROSS'
}

In [18]:
# Replace the "modele" name
df['modele'] = df['modele'].replace(dict_model_corr)

In [19]:
# Homogénéiser le format des colonnes pour les deux DataFrames
df['marque'] = df['marque'].str.upper()
df['finition'] = df['finition'].str.upper()
df['modele'] = df['modele'].str.upper()
df['puissance'] = df['puissance'].astype('Int64')

df_prix_neuf['option_marque_select'] = df_prix_neuf['option_marque_select'].str.upper()
df_prix_neuf['Versions'] = df_prix_neuf['Versions'].str.upper()
df_prix_neuf['source_model'] = df_prix_neuf['source_model'].str.upper()
df_prix_neuf['Version_selected'] = df_prix_neuf['Version_selected'].str.upper()
df_prix_neuf['Portes'] = df_prix_neuf['Portes'].astype('Int64')
df_prix_neuf['option_year_select'] = df_prix_neuf['option_year_select'].astype('Int64')

# Rename columns
df_prix_neuf.rename(columns={"Prix": "prix_neuf", "url" : 'url_prix_neuf'}, inplace=True)

# Homogénéiser le type de boite de vitesse
dict_boite = {'Double embrayage / DCT' : 'Boite de vitesse automatique',
              'Semi-automatique' : 'Boite de vitesse automatique',
              'Automatique': 'Boite de vitesse automatique',
              'Mécanique': 'Boite de vitesse manuelle'
              }
df_prix_neuf['Boite'] = df_prix_neuf['Boite'].replace(dict_boite)
df['transmission'] = df['transmission'].replace(dict_boite)


In [20]:
df_prix_neuf.head()

,source_model,Versions,Portes,Energie,Boite,CO2\n(g/km),prix_neuf,url_prix_neuf,option_marque_select,option_year_select,Version_selected
0,ABARTH 124 SPIDER,II 1.4 TURBO 170,2,Ess.,Boite de vitesse manuelle,146 (nedc),34 500 €,https://www.caradisiac.com/fiches-techniques/m...,ABARTH,2019,ABARTH 124 2E GENERATION SPIDER
1,ABARTH 124 SPIDER,II 1.4 TURBO 170 GT,2,Ess.,Boite de vitesse manuelle,146 (nedc),40 900 €,https://www.caradisiac.com/fiches-techniques/m...,ABARTH,2019,ABARTH 124 2E GENERATION SPIDER
2,ABARTH 124 SPIDER,II 1.4 TURBO 170 GT BVA,2,Ess.,Boite de vitesse automatique,161 (nedc),42 900 €,https://www.caradisiac.com/fiches-techniques/m...,ABARTH,2019,ABARTH 124 2E GENERATION SPIDER
3,ABARTH 124 SPIDER,II 1.4 TURBO 170 TURISMO,2,Ess.,Boite de vitesse manuelle,146 (nedc),37 500 €,https://www.caradisiac.com/fiches-techniques/m...,ABARTH,2019,ABARTH 124 2E GENERATION SPIDER
4,ABARTH 124 SPIDER,II 1.4 TURBO 170 TURISMO BVA,2,Ess.,Boite de vitesse automatique,161 (nedc),39 500 €,https://www.caradisiac.com/fiches-techniques/m...,ABARTH,2019,ABARTH 124 2E GENERATION SPIDER


In [41]:
df_prix_neuf[df_prix_neuf['prix_neuf'] == "NC"]

,source_model,Versions,Portes,Energie,Boite,CO2\n(g/km),prix_neuf,url_prix_neuf,option_marque_select,option_year_select,Version_selected
16935,MG ZS,ELECTRIQUE,5,Elec.,Boite de vitesse automatique,NC,NC,https://www.caradisiac.com/fiches-techniques/m...,MG,2022,MG MG ZS
18716,MINI MINI,III (F55) 1.5 136 COOPER BVA6 5P,5,Ess.,Boite de vitesse automatique,NC,NC,https://www.caradisiac.com/fiches-techniques/m...,MINI,2019,MINI MINI 3 5P
18905,MINI MINI,III (F55) 1.5 136 COOPER BVA6 5P,5,Ess.,Boite de vitesse automatique,NC,NC,https://www.caradisiac.com/fiches-techniques/m...,MINI,2020,MINI MINI 3 5P
19173,MINI MINI,III (F55) 1.5 136 COOPER BVA6 5P,5,Ess.,Boite de vitesse automatique,NC,NC,https://www.caradisiac.com/fiches-techniques/m...,MINI,2021,MINI MINI 3 5P
30348,PEUGEOT 308,1.6 THP 140 PREMIUM BVA 3P,3,Ess.,Boite de vitesse automatique,NC,NC,https://www.caradisiac.com/fiches-techniques/m...,PEUGEOT,2023,PEUGEOT 308


In [21]:
# 1. Merging df with df_prix_neuf to get the listing price. Keys: marque, modele, annee
df_merge_1 = df.merge(df_prix_neuf, how="left", left_on=['marque', 'modele', 'annee'], right_on = ['option_marque_select', 'source_model', 'option_year_select'] )

In [22]:
df.head()

,scraped_at,marque,modele,finition,prix,annee_mise_en_circulation,kilometrage,carburant,transmission,puissance,...,couleur,sellerie,classe_emission,emission_CO2,crit_air,usage_commerciale_anterieure,url_annonce,annee,age_days,age_years
0,2025-04-09,FORD,FORD FIESTA,1.0 ECOBOOST ST-LINE X,13190.0,15.05.2020,69301.0,Essence,Boite de vitesse manuelle,95,...,Gris,Tissu (Sellerie d'origine),EURO 6,NaN,Crit'Air 1,NaN,https://www.autohero.com/fr/ford-fiesta/id/516...,2020,1790.0,4.9
1,2025-04-09,TOYOTA,TOYOTA PROACE,COMBI LONG 1.5 D-4D DYNAMIC,23990.0,29.04.2021,71887.0,Diesel,Boite de vitesse manuelle,120,...,Gris,Tissu (Sellerie d'origine),EURO 6,170.0,Crit'Air 2,Oui,https://www.autohero.com/fr/toyota-pro-ace/id/...,2021,1441.0,3.9
2,2025-04-09,MERCEDES,MERCEDES-BENZ GLA,250 E AMG LINE 8G-DCT,32490.0,23.10.2020,59649.0,Hybride,Boite de vitesse automatique,218,...,Gris,Mi-cuir (Sellerie d'origine),EURO 6,32.0,Crit'Air 1,Oui,https://www.autohero.com/fr/mercedes-benz-gla/...,2020,1629.0,4.5
3,2025-04-09,BMW,BMW X1,SDRIVE18I XLINE DKG7,27890.0,05.05.2021,37869.0,Essence,Boite de vitesse automatique,136,...,Noir,Mi-cuir (Sellerie d'origine),EURO 6,148.0,Crit'Air 1,Non,https://www.autohero.com/fr/bmw-x-1/id/490f203...,2021,1435.0,3.9
4,2025-04-09,PEUGEOT,PEUGEOT 3008,1.5 BLUE-HDI CROSSWAY EAT8,19090.0,31.12.2019,58958.0,Diesel,Boite de vitesse automatique,130,...,Blanc,Mi-cuir (Sellerie d'origine),EURO 6,98.0,Crit'Air 2,Oui,https://www.autohero.com/fr/peugeot-3008/id/df...,2019,1926.0,5.3


In [23]:
# Concaténer puissance avec la finition
df_merge_1['finition_puiss'] = df_merge_1['finition'].astype(str) + " " + df_merge_1['puissance'].astype(str) + " CV"

# Concaténer "Versions" avec "Version_selected"
df_merge_1['Version_finale'] = df_merge_1['Version_selected'].astype(str) + " " + df_merge_1['Versions'].astype(str)

In [24]:
df_merge_1

,scraped_at,marque,modele,finition,prix,annee_mise_en_circulation,kilometrage,carburant,transmission,puissance,...,Energie,Boite,CO2\n(g/km),prix_neuf,url_prix_neuf,option_marque_select,option_year_select,Version_selected,finition_puiss,Version_finale
0,2025-04-09,FORD,FORD FIESTA,1.0 ECOBOOST ST-LINE X,13190.0,15.05.2020,69301.0,Essence,Boite de vitesse manuelle,95,...,Ess.,Boite de vitesse manuelle,114 (wltp),22 790 €,https://www.caradisiac.com/fiches-techniques/m...,FORD,2020,FORD FIESTA 6,1.0 ECOBOOST ST-LINE X 95 CV,FORD FIESTA 6 VI (2) 1.0 ECOBOOST 125 MHEV ST-...
1,2025-04-09,FORD,FORD FIESTA,1.0 ECOBOOST ST-LINE X,13190.0,15.05.2020,69301.0,Essence,Boite de vitesse manuelle,95,...,Ess.,Boite de vitesse manuelle,114 (wltp),23 890 €,https://www.caradisiac.com/fiches-techniques/m...,FORD,2020,FORD FIESTA 6,1.0 ECOBOOST ST-LINE X 95 CV,FORD FIESTA 6 VI (2) 1.0 ECOBOOST 125 MHEV ST-...
2,2025-04-09,FORD,FORD FIESTA,1.0 ECOBOOST ST-LINE X,13190.0,15.05.2020,69301.0,Essence,Boite de vitesse manuelle,95,...,Ess.,Boite de vitesse automatique,149 (wltp),21 100 €,https://www.caradisiac.com/fiches-techniques/m...,FORD,2020,FORD FIESTA 6,1.0 ECOBOOST ST-LINE X 95 CV,FORD FIESTA 6 VI 1.0 ECOBOOST 100 5CV CONNECT ...
3,2025-04-09,FORD,FORD FIESTA,1.0 ECOBOOST ST-LINE X,13190.0,15.05.2020,69301.0,Essence,Boite de vitesse manuelle,95,...,Ess.,Boite de vitesse automatique,149 (wltp),20 600 €,https://www.caradisiac.com/fiches-techniques/m...,FORD,2020,FORD FIESTA 6,1.0 ECOBOOST ST-LINE X 95 CV,FORD FIESTA 6 VI 1.0 ECOBOOST 100 5CV S&S CONN...
4,2025-04-09,FORD,FORD FIESTA,1.0 ECOBOOST ST-LINE X,13190.0,15.05.2020,69301.0,Essence,Boite de vitesse manuelle,95,...,Ess.,Boite de vitesse automatique,150 (wltp),21 650 €,https://www.caradisiac.com/fiches-techniques/m...,FORD,2020,FORD FIESTA 6,1.0 ECOBOOST ST-LINE X 95 CV,FORD FIESTA 6 VI 1.0 ECOBOOST 100 5CV S&S TITA...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189591,2025-04-09,RENAULT,RENAULT SCENIC,1.3 TCE SL LIMITED,19990.0,20.04.2021,6321.0,Essence,Boite de vitesse manuelle,140,...,Dies.,Boite de vitesse manuelle,141 (wltp),37 300 €,https://www.caradisiac.com/fiches-techniques/m...,RENAULT,2021,RENAULT SCENIC 4,1.3 TCE SL LIMITED 140 CV,RENAULT SCENIC 4 IV 1.7 DCI 150 8CV BLUE SL BL...
189592,2025-04-09,RENAULT,RENAULT SCENIC,1.3 TCE SL LIMITED,19990.0,20.04.2021,6321.0,Essence,Boite de vitesse manuelle,140,...,Dies.,Boite de vitesse automatique,153 (wltp),39 300 €,https://www.caradisiac.com/fiches-techniques/m...,RENAULT,2021,RENAULT SCENIC 4,1.3 TCE SL LIMITED 140 CV,RENAULT SCENIC 4 IV 1.7 DCI 150 8CV BLUE SL BL...
189593,2025-04-09,RENAULT,RENAULT SCENIC,1.3 TCE SL LIMITED,19990.0,20.04.2021,6321.0,Essence,Boite de vitesse manuelle,140,...,Dies.,Boite de vitesse automatique,156 (wltp),37 300 €,https://www.caradisiac.com/fiches-techniques/m...,RENAULT,2021,RENAULT SCENIC 4,1.3 TCE SL LIMITED 140 CV,RENAULT SCENIC 4 IV 1.7 DCI 150 BLUE INTENS BL...
189594,2025-04-09,RENAULT,RENAULT SCENIC,1.3 TCE SL LIMITED,19990.0,20.04.2021,6321.0,Essence,Boite de vitesse manuelle,140,...,Dies.,Boite de vitesse manuelle,153 (wltp),33 300 €,https://www.caradisiac.com/fiches-techniques/m...,RENAULT,2021,RENAULT SCENIC 4,1.3 TCE SL LIMITED 140 CV,RENAULT SCENIC 4 IV 1.7 DCI 150 BUSINESS


In [25]:
# 2. Sélectionner les versions les plus proches du cible
# Création d'un système de notation pour évaluer la proximité entre les versions
# finition_puiss & Versions, transmission & Boite, nb_porte vs Portes

# Calculer le score de proximité entre la finition_puissance et Version_finale
df_merge_1['note_version_commune'] = df_merge_1.apply(
	lambda x: len(set(x['finition_puiss'].split()) & set(x['Version_finale'].split())) if pd.notnull(x['finition_puiss']) and pd.notnull(x['Version_finale']) else 0,
	axis=1
)

# Calculer le score de proximité entre la transmission et Boite
df_merge_1['note_transmission_commune'] = df_merge_1.apply(
	lambda x: 1 if pd.notnull(x['transmission']) and pd.notnull(x['Boite']) and x['transmission'] == x['Boite'] else 0,
	axis=1
)

# Calculer le score de proximité entre le nombre de portes et Portes
df_merge_1['note_nb_porte_commune'] = df_merge_1.apply(
	lambda x: 1 if pd.notnull(x['nb_porte']) and pd.notnull(x['Portes']) and x['nb_porte'] == x['Portes'] else 0,
	axis=1
)

# Calculer le score total
df_merge_1['note_commune_total'] = df_merge_1['note_version_commune'] +df_merge_1['note_transmission_commune'] + df_merge_1['note_nb_porte_commune']

In [26]:
# Garder les lignes avec les scores les plus élevés pour chaque combinaison de marque, modele et annee
df_merge_1['max_note'] = df_merge_1.groupby(['marque', 'modele', 'annee'])['note_commune_total'].transform('max')
df_merge_2 = df_merge_1[df_merge_1['note_commune_total'] == df_merge_1['max_note']]
df_merge_2.sort_values(by=['marque', 'modele', 'annee'], ascending=[True, True, True], inplace=True)
df_merge_2.reset_index(drop=True, inplace=True)

C:\Users\Hong-CuongLE\AppData\Local\Temp\ipykernel_18484\3568166748.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merge_2.sort_values(by=['marque', 'modele', 'annee'], ascending=[True, True, True], inplace=True)


In [27]:
df_merge_2.shape

(2880, 42)

In [26]:
df_merge_2[df_merge_2['note_commune_total']== 0]

,scraped_at,marque,modele,finition,prix,annee_mise_en_circulation,kilometrage,carburant,transmission,puissance,...,option_marque_select,option_year_select,Version_selected,finition_puiss,Version_finale,note_version_commune,note_transmission_commune,note_nb_porte_commune,note_commune_total,max_note
359,2025-04-09,CITROEN,C4 PICASSO,(2) 1.2 PURETECH SHINE BV6,10690.0,17.01.2018,87749.0,Essence,Boite de vitesse manuelle,130,...,NaN,<NA>,NaN,(2) 1.2 PURETECH SHINE BV6 130 CV,nan nan,0,0,0,0,0.0
360,2025-04-09,CITROEN,C4 PICASSO,(2) 1.2 PURETECH FEEL BV6,10790.0,19.11.2018,44552.0,Essence,Boite de vitesse manuelle,130,...,NaN,<NA>,NaN,(2) 1.2 PURETECH FEEL BV6 130 CV,nan nan,0,0,0,0,0.0
361,2025-04-09,CITROEN,C4 PICASSO,(2) 1.2 PURETECH SHINE BV6,10690.0,17.01.2018,87749.0,Essence,Boite de vitesse manuelle,130,...,NaN,<NA>,NaN,(2) 1.2 PURETECH SHINE BV6 130 CV,nan nan,0,0,0,0,0.0
362,2025-04-09,CITROEN,C4 PICASSO,(2) 1.2 PURETECH SHINE EAT6,11690.0,24.04.2019,65767.0,Essence,Boite de vitesse automatique,130,...,NaN,<NA>,NaN,(2) 1.2 PURETECH SHINE EAT6 130 CV,nan nan,0,0,0,0,0.0
363,2025-04-09,CITROEN,C4 PICASSO,1.2 PURETECH SHINE BV6,10590.0,31.05.2019,87702.0,Essence,Boite de vitesse manuelle,130,...,NaN,<NA>,NaN,1.2 PURETECH SHINE BV6 130 CV,nan nan,0,0,0,0,0.0
1165,2025-04-09,FORD,KA+,1.2 TI-VCT ESSENTIAL,7390.0,20.12.2017,77468.0,Essence,Boite de vitesse manuelle,70,...,NaN,<NA>,NaN,1.2 TI-VCT ESSENTIAL 70 CV,nan nan,0,0,0,0,0.0
1380,2025-04-09,KIA,PROCEED,1.0 T-GDI ISG GT-LINE BV6,10790.0,16.03.2017,84553.0,Essence,Boite de vitesse manuelle,120,...,NaN,<NA>,NaN,1.0 T-GDI ISG GT-LINE BV6 120 CV,nan nan,0,0,0,0,0.0
1749,2025-04-09,OPEL,CROSSLAND,1.2 TURBO INNOVATION AUTO,10890.0,24.08.2017,83454.0,Essence,Boite de vitesse automatique,110,...,NaN,<NA>,NaN,1.2 TURBO INNOVATION AUTO 110 CV,nan nan,0,0,0,0,0.0
1750,2025-04-09,OPEL,CROSSLAND,1.2 TURBO INNOVATION,11390.0,06.08.2018,61644.0,Essence,Boite de vitesse manuelle,110,...,NaN,<NA>,NaN,1.2 TURBO INNOVATION 110 CV,nan nan,0,0,0,0,0.0
1751,2025-04-09,OPEL,CROSSLAND,1.2 TURBO ULTIMATE AUTOMATIQUE,12590.0,30.04.2018,57620.0,Essence,Boite de vitesse automatique,110,...,NaN,<NA>,NaN,1.2 TURBO ULTIMATE AUTOMATIQUE 110 CV,nan nan,0,0,0,0,0.0


In [27]:
df_prix_neuf[(df_prix_neuf['source_model'].str.contains("KA+", na=False))]

,source_model,Versions,Portes,Energie,Boite,CO2\n(g/km),prix_neuf,url_prix_neuf,option_marque_select,option_year_select,Version_selected
12414,FORD KA+,1.3 COLLECTION,3,Ess.,Boite de vitesse manuelle,154 (nedc),10 600 €,https://www.caradisiac.com/fiches-techniques/m...,FORD,2017,FORD KA
12415,FORD KA+,1.3 ELANCE,3,Ess.,Boite de vitesse manuelle,154 (nedc),12 540 €,https://www.caradisiac.com/fiches-techniques/m...,FORD,2017,FORD KA
12416,FORD KA+,1.3 OBSESSION,3,Ess.,Boite de vitesse manuelle,154 (nedc),9 700 €,https://www.caradisiac.com/fiches-techniques/m...,FORD,2017,FORD KA
12417,FORD KA+,1.3 OBSESSION PACK,3,Ess.,Boite de vitesse manuelle,154 (nedc),10 400 €,https://www.caradisiac.com/fiches-techniques/m...,FORD,2017,FORD KA
12418,FORD KA+,1.3 ORIGINAL,3,Ess.,Boite de vitesse manuelle,NC,8 900 €,https://www.caradisiac.com/fiches-techniques/m...,FORD,2017,FORD KA
...,...,...,...,...,...,...,...,...,...,...,...
54959,KA+,1.3 COLLECTION,3,Ess.,Boite de vitesse manuelle,154 (nedc),10 600 €,https://www.caradisiac.com/fiches-techniques/m...,FORD,2018,FORD KA
54960,KA+,1.3 ELANCE,3,Ess.,Boite de vitesse manuelle,154 (nedc),12 540 €,https://www.caradisiac.com/fiches-techniques/m...,FORD,2018,FORD KA
54961,KA+,1.3 OBSESSION,3,Ess.,Boite de vitesse manuelle,154 (nedc),9 700 €,https://www.caradisiac.com/fiches-techniques/m...,FORD,2018,FORD KA
54962,KA+,1.3 OBSESSION PACK,3,Ess.,Boite de vitesse manuelle,154 (nedc),10 400 €,https://www.caradisiac.com/fiches-techniques/m...,FORD,2018,FORD KA


## 4. Traitement des valeurs manquantes

In [ ]:
# Liste des missing values
missing_values = df.isnull().sum()
print(missing_values)

In [ ]:
# Supprimer les lignes où le prix est manquant
df.dropna(subset=["prix"], inplace=True)
missing_values = df.isnull().sum()
print(missing_values)

Il reste 256 lignes où l'émission de CO2 et le top usage_commerciale_anterieure sont vides  

In [ ]:
df_missing = df[(df["emission_CO2"].isnull()) | 
                (df["usage_commerciale_anterieure"].isnull())
                ]
print(df_missing.shape)
df_missing.head()

In [ ]:
# Regarder si emission_CO2 est renseignée quelque part pour les voitures de même modèle et finition
# La liste des valeurs uniques de emission_CO2 pour chaque modèle et finition
df_emission_CO2_stats = df[df["emission_CO2"].notnull()].groupby(["annee", "modele", "finition"]) \
    .agg({"emission_CO2": ["nunique", "min", "max", "mean"]}) \
    .reset_index()

# Rename the column ('annee', '') to 'annee'
df_emission_CO2_stats.columns = ['annee', 'modele', 'finition', 'nunique', 'min', 'max', 'mean']

# Sort the dataframe by the number of unique values in emission_CO2
df_emission_CO2_stats = df_emission_CO2_stats.sort_values("nunique", ascending=False)

# Si emission_CO2 est missing, on prend la moyenne de emission_CO2 pour l'année, le modèle et la finition
df_missing = df_missing.merge(df_emission_CO2_stats[["annee", "modele", "finition", "mean"]],
                               how="left",
                               on=["annee", "modele", "finition"])
df_missing.rename(columns={"mean": "emission_CO2_mean"}, inplace=True)
df_missing["emission_CO2"] = df_missing["emission_CO2"].fillna(df_missing["emission_CO2_mean"])

In [ ]:
# Nombre de missing values après le remplissage
missing_values = df_missing.isnull().sum()
print(missing_values)

In [ ]:
# Sur les missing values restants, on remplace emission_CO2 par la moyenne de emission_CO2 pour le modèle
df_emission_CO2_stats_2= df[df["emission_CO2"].notnull()].groupby(["modele", "finition"]) \
                                .agg({"emission_CO2": ["nunique", "min", "max", "mean"]}) \
                                .reset_index()

# Rename the column ('annee', '') to 'annee'
df_emission_CO2_stats_2.columns = ['modele', 'finition', 'nunique', 'min', 'max', 'mean']

# Imputer les valeurs manquantes restantes de emission_CO2 
# par la moyenne de emission_CO2 pour le modèle et la finition
df_missing = df_missing.merge(df_emission_CO2_stats_2[["modele", "finition", "mean"]],
                               how="left",
                               on=["modele", "finition"])
df_missing.rename(columns={"mean": "emission_CO2_mean_2"}, inplace=True)
df_missing["emission_CO2"] = df_missing["emission_CO2"].fillna(df_missing["emission_CO2_mean_2"])
df_missing.drop(columns=["emission_CO2_mean", "emission_CO2_mean_2"], inplace=True)

# Nombre de missing values après le remplissage
missing_values = df_missing.isnull().sum()
print(missing_values)

Sur les cas qui restent, imputer comme suit:
- emission_C02: moyenne d'émission par marque et carburant
- usage_commerciale_anterieure: utiliser mode

In [ ]:
# Liste des colonnes alphanumériques
alphanumeric_columns = df.select_dtypes(include=[object]).columns.tolist()
print(alphanumeric_columns)

In [ ]:
# Les valeurs uniques de chaque colonne alphanumérique
# exclure les colonnes dans list_of_columns_to_drop
# afficher les 10 valeurs les plus fréquentes
list_of_columns_to_drop = ['scraped_at', 'url_annonce', 'annee_mise_en_circulation']
for col in alphanumeric_columns:
    if col in list_of_columns_to_drop:
        continue    
    print(f"Colonne: {col}")
    value_counts = df[col].value_counts(normalize=True, dropna=False).head(10)
    absolute_counts = df[col].value_counts(dropna=False).head(10)
    percentages = (value_counts * 100).round(2).astype(str) + " %"
    print(pd.DataFrame({'Valeur': absolute_counts.index, 
                        'Fréquence': absolute_counts.values, 
                        'Pourcentage': percentages.values
                        }
                        ).reset_index(drop=True)
        )
    print("===" * 20)

In [ ]:
# Histogramme

In [ ]:
df.describe(include = "object")

In [ ]:
df.describe(include="number")

In [ ]:
# Check for missing values
missing_values = df.isnull().sum()
print(missing_values)

In [ ]:
df_missing_prix = df[df["prix"].isnull()]
df_missing_prix.head()

In [ ]:
count = df["modele"].value_counts()

In [ ]:
count.reset_index()

In [ ]:
df.shape